In [8]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext ,SQLContext,Row,HiveContext
import sys
import re
from pyspark.sql import Row

conf = SparkConf().setAppName("Apache Log Analyzer")
#sc = SparkContext(conf=conf)

s='64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846'
print (s)

64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846


In [12]:
'''
Creating a function to read each element using Regex. This module will be passed later on in the notebook.
'''
def log_line(line):
    patrn = '^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\d+)'
    mtch = re.search(patrn,line)
    if mtch is None:
        pass
    else :
        ip = mtch.group(1)
        client = mtch.group(2)
        user = mtch.group(3)
        date_time = mtch.group(4)
        mode = mtch.group(5)
        target = mtch .group(6)
        protocol = mtch.group(7)
        res_code = mtch.group(8)
        size = mtch.group(9)
        return Row(ip,client,user,date_time,mode,target,protocol,res_code,size)
print (log_line(s))
    

<Row(64.242.88.10, -, -, 07/Mar/2004:16:05:49 -0800, GET, /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables, HTTP/1.1, 401, 12846)>


In [13]:
# Reading source file form HDFS and creating RDD

logFileRdd = sc.textFile("/user/data/apache_log.txt")
logFileRdd.take(5)

[u'64.242.88.10 - - [07/Mar/2004:16:05:49 -0800] "GET /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables HTTP/1.1" 401 12846',
 u'64.242.88.10 - - [07/Mar/2004:16:06:51 -0800] "GET /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2 HTTP/1.1" 200 4523',
 u'64.242.88.10 - - [07/Mar/2004:16:10:02 -0800] "GET /mailman/listinfo/hsdivision HTTP/1.1" 200 6291',
 u'64.242.88.10 - - [07/Mar/2004:16:11:58 -0800] "GET /twiki/bin/view/TWiki/WikiSyntax HTTP/1.1" 200 7352',
 u'64.242.88.10 - - [07/Mar/2004:16:20:55 -0800] "GET /twiki/bin/view/Main/DCCAndPostFix HTTP/1.1" 200 5253']

In [30]:
parseLogFileRdd = logFileRdd.map(log_line).cache()
parseLogFileRdd.take(5)

[<Row(64.242.88.10, -, -, 07/Mar/2004:16:05:49 -0800, GET, /twiki/bin/edit/Main/Double_bounce_sender?topicparent=Main.ConfigurationVariables, HTTP/1.1, 401, 12846)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:06:51 -0800, GET, /twiki/bin/rdiff/TWiki/NewUserTemplate?rev1=1.3&rev2=1.2, HTTP/1.1, 200, 4523)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:10:02 -0800, GET, /mailman/listinfo/hsdivision, HTTP/1.1, 200, 6291)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:11:58 -0800, GET, /twiki/bin/view/TWiki/WikiSyntax, HTTP/1.1, 200, 7352)>,
 <Row(64.242.88.10, -, -, 07/Mar/2004:16:20:55 -0800, GET, /twiki/bin/view/Main/DCCAndPostFix, HTTP/1.1, 200, 5253)>]

In [33]:
# Calculate rescode count
#parseLogFileRdd.map(lambda x : (int(x[7]),1)).take(10)
resCodeCnt = (parseLogFileRdd.map(lambda x : (int(x[7]),1)).reduceByKey(lambda a,b:a+b))
resCodeCnt.take(10)
#parseLogFileRdd
print ("Response Code Counts: %s" % (resCodeCnt.take(10)))

Response Code Counts: [(200, 1272), (404, 5), (302, 6), (401, 123)]


In [44]:
# Calculate  content size .
contentSize = parseLogFileRdd.map(lambda x: int(x[8]))
print ("Avg Content size :  %i  Min Content size : %i  Max Content size :  %i" % (contentSize.reduce(lambda a,b : a+b)/contentSize.count(),
       contentSize.min(),contentSize.max()))
#contentSize.take(10)

Avg Content size :  7775  Min Content size : 0  Max Content size :  138789


In [56]:
# Any IPAddress that has accessed the server more than 10 times.
ipCnt = parseLogFileRdd.map(lambda x : str(x[0])).map(lambda x : (x,1)).reduceByKey(lambda a,b: a+b)\
        .filter(lambda x : x[1]>20).map(lambda x : x[0]).take(20)
#ipCnt.take(10)
print ("IpAddresses that have accessed more then 20 times: %s" % (ipCnt))

IpAddresses that have accessed more then 20 times: ['10.0.0.153', 'ip68-228-43-49.tc.ph.cox.net', 'h24-70-69-74.ca.shawcable.net', 'h24-71-236-129.ca.shawcable.net', 'cr020r01-3.sac.overture.com', 'ts04-ip92.hevanet.com', 'market-mail.panduit.com', '64.242.88.10']


In [62]:
ipCnt = parseLogFileRdd.map(lambda x : str(x[0])).map(lambda x : (x,1)).reduceByKey(lambda a,b: a+b)\
        .filter(lambda x : x[1]>20).take(20)
ipCntDict = {}
print ("Converting into  python dictionary")
for x in ipCnt:
    ipCntDict[x[0]]=x[1]
print (ipCntDict)
print("Max time an IP Hit the server ")
print( [(x,y) for x,y in ipCntDict.items() if y == max(ipCntDict.values())])
    


Converting into  python dictionary
{'10.0.0.153': 188, 'ts04-ip92.hevanet.com': 28, 'cr020r01-3.sac.overture.com': 44, 'h24-70-69-74.ca.shawcable.net': 32, 'market-mail.panduit.com': 29, 'ip68-228-43-49.tc.ph.cox.net': 22, '64.242.88.10': 452, 'h24-71-236-129.ca.shawcable.net': 36}
Max time an IP Hit the server 
[('64.242.88.10', 452)]
